In [4]:
!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0

  Using cached grpcio-1.42.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.0 MB)
INFO: pip is looking at multiple versions of tensorboard to determine which version is compatible with other requirements. This could take a while.
  Using cached tensorboard-2.15.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached tensorflow-2.15.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached tensorflow-2.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
  Using cached tensorflow-2.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tensorboard-2.14.1-py3-none-any.whl.metadata (1.7 kB)
  Using cached tensorflow_estimator-2.14.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached keras-2.14.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached tensorboard-2.14.0-py3-none-any.whl.metadata (1.8 kB)
INFO: pip is still looking at multiple versions of tensorboard to d

In [5]:
!pip install keras-image-helper

In [7]:
import grpc

import tensorflow as tf

from tensorflow import keras 
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [11]:
model = keras.models.load_model('./saved_models/ResNet_lr_0.10000000149011612_size_NA_dropout_0.5_01_0.953.h5')

In [12]:
tf.saved_model.save(model, './location_classifier')

INFO:tensorflow:Assets written to: /home/minasonbol/Documents/study/mleng-zoomcamp/capstone_01/capstone/code/location_classifier/assets


INFO:tensorflow:Assets written to: /home/minasonbol/Documents/study/mleng-zoomcamp/capstone_01/capstone/code/location_classifier/assets


In [15]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [16]:
from keras_image_helper import create_preprocessor

In [31]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [32]:
url = 'https://raw.githubusercontent.com/el-grudge/mleng-zoomcamp/main/capstone_01/cairo_frame1050.jpg'
X = preprocessor.from_url(url)

In [33]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [37]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'location_classifier'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_38'].CopyFrom(np_to_protobuf(X))

In [38]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [43]:
preds = pb_response.outputs['dense_18'].float_val

In [44]:
classes = [
    'cairo',
    'moscow',
    'pairs'
]

In [46]:
dict(zip(classes, preds))

{'cairo': -1.7488744258880615,
 'moscow': -5.521413326263428,
 'pairs': -30.53763198852539}